In [3]:
import polars as pl
from scipy.sparse import coo_array, diags
from sksparse.cholmod import cholesky

from benchmarks.utils import mock_snakemake

DF_CUTOFF = 0

if "snakemake" not in globals():  # noqa: F821
    snakemake = mock_snakemake("compute_ptdf")

lines = pl.read_parquet(snakemake.input.lines)
buses = pl.concat([lines["from_bus"], lines["to_bus"]]).unique()
L = len(lines)
N = len(buses)
lines.sort("from_bus")

line_id,from_bus,to_bus,reactance,line_rating,is_leaf
i64,i64,i64,f64,f64,bool
1,1,2410,0.000219,1.75,true
3280,3,3972,0.027643,0.64,false
3285,3,3971,0.027293,0.64,false
3288,3,3976,0.018766,0.3,true
3283,3,3975,0.052057,0.8,false
…,…,…,…,…,…
10815,8854,8855,0.115871,0.940328,false
10819,8860,8862,0.020603,5.29,false
10818,8861,8862,0.034818,3.13,false


In [4]:
# Step 1: Renumber buses and lines such that they go from 1 to N contiguously.
lines = lines.sort("line_id")
lines = lines.rename({"line_id": "old_line_id"}).with_row_index(name="line_id")
line_mapping = lines.select("line_id", "old_line_id")
lines = lines.drop("old_line_id")

bus_mapping = (
    buses.sort().to_frame().with_row_index(name="new_bus").rename({"from_bus": "bus"})
)
lines = (
    lines.join(bus_mapping.rename({"bus": "from_bus"}), on="from_bus")
    .drop("from_bus")
    .rename({"new_bus": "from_bus"})
    .join(bus_mapping.rename({"bus": "to_bus"}), on="to_bus")
    .drop("to_bus")
    .rename({"new_bus": "to_bus"})
)
lines.sort("from_bus")

line_id,reactance,line_rating,is_leaf,from_bus,to_bus
u32,f64,f64,bool,u32,u32
0,0.000219,1.75,true,0,1982
2390,0.027643,0.64,false,1,2960
2393,0.052057,0.8,false,1,2962
2394,0.027293,0.64,false,1,2959
2397,0.018766,0.3,true,1,2963
…,…,…,…,…,…
8119,0.115871,0.940328,false,6461,6462
8121,0.020603,5.29,false,6464,6466
8120,0.034818,3.13,false,6465,6466


In [5]:
# Step 2: Form an admittance matrix.
lines = lines.with_columns(suscep=1 / lines["reactance"])
off_diagonal = pl.concat(
    [
        lines.select(
            pl.col("from_bus").alias("i"),
            pl.col("to_bus").alias("j"),
            -pl.col("suscep"),
        ),
        lines.select(
            pl.col("to_bus").alias("i"),
            pl.col("from_bus").alias("j"),
            -pl.col("suscep"),
        ),
    ]
)
# TODO double check
diagonal = (
    pl.concat(
        [
            lines.select("suscep", i=pl.col("from_bus")),
            lines.select("suscep", i=pl.col("to_bus")),
        ]
    )
    .group_by("i")
    .agg(pl.sum("suscep"))
    .select("i", pl.col("i").alias("j"), "suscep")
)
assert len(diagonal) == N, "Diagonal entries should match the number of buses."
admittance_matrix_df = pl.concat([off_diagonal, diagonal])
admittance_matrix_df

i,j,suscep
u32,u32,f64
0,1982,-4566.772343
17,1982,-133.59381
1982,1983,-61.96631
1322,1983,-2133.621717
1321,1983,-243.643906
…,…,…
1703,1703,21.583459
4320,4320,393.555103
5502,5502,163.231729


In [6]:
# Step 3: Drop last row and column (slack bus).
admittance_matrix_df = admittance_matrix_df.filter(
    pl.col("i") < (N - 1), pl.col("j") < (N - 1)
)
admittance_matrix = coo_array(
    (
        admittance_matrix_df["suscep"].to_numpy(),
        (
            admittance_matrix_df["i"].cast(pl.Float64).to_numpy(),
            admittance_matrix_df["j"].cast(pl.Float64).to_numpy(),
        ),
    ),
    shape=(N - 1, N - 1),
)
admittance_matrix

<COOrdinate sparse array of dtype 'float64'
	with 22714 stored elements and shape (6472, 6472)>

In [7]:
# Step 4: Inverse the admittance matrix to get voltage angles.
# We use the sparse Cholesky factorization since, based on my tests, this is much faster than using
# scipy.linalg.inv, scipy.sparse.linalg.inv, scipy.linag.pinv, scipy.sparse.linalg.spsolve
factor = cholesky(admittance_matrix.tocsc())
voltage_angles = factor.inv()
voltage_angles

/tmp/ipykernel_2058273/646270145.py:5: CholmodTypeConversionWarning: array contains 32 bit integers; but 64 bit integers are needed; slowing down due to converting
  voltage_angles = factor.inv()


<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 41886784 stored elements and shape (6472, 6472)>

In [8]:
adjacency_matrix_T = pl.concat(
    [
        lines.select(
            i=pl.col("line_id"),
            j=pl.col("from_bus"),
            val=pl.lit(1),
        ).filter(pl.col("j") < N - 1),  # Exclude slack bus,
        lines.select(
            i=pl.col("line_id"),
            j=pl.col("to_bus"),
            val=pl.lit(-1),
        ).filter(pl.col("j") < N - 1),  # Exclude slack bus
    ]
)
adjacency_matrix_T = coo_array(
    (
        adjacency_matrix_T["val"].to_numpy(),
        (adjacency_matrix_T["i"].to_numpy(), adjacency_matrix_T["j"].to_numpy()),
    ),
    shape=(len(lines), N - 1),
)
susceptance_matrix = diags(lines.sort("line_id")["suscep"].to_numpy())
power_flow = susceptance_matrix @ coo_array(adjacency_matrix_T) @ voltage_angles
power_flow = power_flow.tocoo()
power_flow_df = pl.DataFrame(
    {
        "injection": power_flow.col,
        "line": power_flow.row + 1,
        "factor": power_flow.data,
    }
)
power_flow_df

injection,line,factor
i32,i32,f64
0,1,1.0
6471,2,-0.001003
6470,2,0.000805
6469,2,0.0008
6468,2,0.00078
…,…,…
4,8124,-0.000008
3,8124,0.000009
2,8124,-9.4857e-7


In [9]:
if DF_CUTOFF is not None:
    power_flow_df = power_flow_df.filter(pl.col("factor").abs() > DF_CUTOFF)
power_flow_df

injection,line,factor
i32,i32,f64
0,1,1.0
6471,2,-0.001003
6470,2,0.000805
6469,2,0.0008
6468,2,0.00078
…,…,…
4,8124,-0.000008
3,8124,0.000009
2,8124,-9.4857e-7


In [10]:
# Unmap buses to original IDs.
power_flow_df_unmapped = (
    power_flow_df.join(bus_mapping.rename({"new_bus": "injection"}), on="injection")
    .drop("injection")
    .rename({"bus": "injection"})
    .join(line_mapping.rename({"line_id": "line"}), on="line")
    .drop("line")
    .rename({"old_line_id": "line"})
)
power_flow_df_unmapped

factor,injection,line
f64,i64,i64
1.0,1,2
-0.001003,8867,3
0.000805,8866,3
0.0008,8865,3
0.00078,8864,3
…,…,…
4.6889e-7,7,10821
-5.8063e-7,6,10821
4.9185e-8,4,10821


In [11]:
power_flow_df_unmapped.write_parquet(snakemake.output[0])
snakemake.output[0]

'/data/home/machstg/pyoframe/benchmarks/input_data/energy_planning/intermediary_data_inputs/power_transfer_dist_facts.parquet'